# Deep Learning - DQN 
### Done by: Nicolas CHAHINE - nicolas.chahine@telecom-paris.fr

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!apt-get install --no-install-recommends ffmpeg && pip install ffmpeg scikit-video

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
     |████████████████████████████████| 2.3MB 4.9MB/s 
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084 sha256=b0c2f89864eaeeb1d49c11956e1f4b232caaffb418d0e7fa09aaa1151c691739
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo 
# skvideo.setFFmpegPath('C:/FFmpeg/ffmpeg-4.1-win64-static/bin/')
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning
### Done by: Nicolas CHAHINE - nicolas.chahine@telecom-paris.fr

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

**Answer 1:** The epsilon parameter is essential to modelize the exploration - exploitation dilemma. On one hand, a random action is taken by the agent with a chance epsilon (which contributes in the exploration). On the other hand, the agent takes an action according to the current policy for a probability 1-epsilon (which favors exploitation). Therefore, with a probability epsilon, we try to discover the outcome of actions chosen randomly, whereas with probability 1-epsilon we choose the best action at the given state to maximize the expected reward. This tradeoff favors the learning of the agent over time; with the Exploration process we make sure that we have chosen the right policy because we discover every possible outcome and the Exploitation will maximize the reward along the way.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**Answer 2:** The board represents the rewards in the NxN cells of the island and the position is used to save the position of the rat before taking the action. In more details, we can explain the use of these arrays as follows: 
- The init method is passed a parameter grid_size, on which we add a value of 4 and construct the 2d arrays position and board of size (grid_size,grid_size). The 4 that has been added is a 2 cell padding((left, right) \ (up,bottom)) for the padding on the edges. Because it is just a padding, in the rest of the code the rat is not allowed to go there: the position values are set to -1 and the conditions in the act method ensure that this is a restricted area.
- When the game starts, we have seen that there is a call to the env.reset() method. In this method, some values in the board are set to 0.5 and some to -1 randomly from a bernouilli distribution with probability equal to temperature(and the overlap is taken as 0.5). The value of the board on the current position is set to 0. 
- In the act method, the current (x,y) coordinates are stored in position. When the rat takes an action and arrives to a new (x,y) coordinate, position is not updated(its purpose is to store the old coordinates). The board value is set to 0 in the new position (symbolising that the rat has eaten the cheese or the poison if available, but it can eat it only once at a specific position).
- The 5x5 patch surrounding the final position of the rat and containing the board and position values ( the surrounding rewards and the previous position) are passed as a state tensor.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            # The agent takes an action
            action = agent.act(state)

            # get the next state and the reward
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

         # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/12.0. Average score (-2.0)
Win/lose count 13.0/14.0. Average score (-1.5)
Win/lose count 10.5/15.0. Average score (-2.5)
Win/lose count 8.0/15.0. Average score (-3.625)
Win/lose count 8.5/14.0. Average score (-4.0)
Win/lose count 10.0/10.0. Average score (-3.3333333333333335)
Win/lose count 14.0/13.0. Average score (-2.7142857142857144)
Win/lose count 7.5/6.0. Average score (-2.1875)
Win/lose count 14.5/13.0. Average score (-1.7777777777777777)
Win/lose count 12.5/8.0. Average score (-1.15)
Final score: -1.15


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer 5:**
- $$Q^\pi(s,a)=E_{(s_1,a_1,....,s_T,a_T) \sim p^\pi}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $$
$$Q^\pi(s,a)=E_{(s_1,a_1,....,s_T,a_T) \sim p^\pi}[r(s_0,a_0) + \sum_{t=1}^T \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$$
$$=r(s,a)+E_{(s_1,a_1,....,s_T,a_T) \sim p^\pi}[\sum_{t=1}^T \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$$
$$=r(s,a)+E_{(s',a') \sim p(.| s,a), \pi(.|s')}\{E_{(s_2,a_2,....,s_T,a_T) \sim p^\pi}[\sum_{t=1}^T \gamma^{t}r(s_{t},a_{t})|s_1=s',a_1=a']\}$$
$$=E_{(s',a') \sim p(.| s,a), \pi(.|s')}\{r(s,a)+\gamma E_{(s_2,a_2,....,s_T,a_T) \sim p^\pi}[r(s_{1},a_{1})+\gamma^2 r(s_2,a_2).....\gamma^{T-1}r(s_T,a_T)|s_1=s',a_1=a']\}$$
$$=E_{(s',a') \sim p(.| s,a), \pi(.|s')}[r(s,a)+\gamma Q^{\pi}(s',a')]$$
The last equality is true because T tends to $\infty$, hence the first equation holds.

- For the optimal policy $\pi^*$, we have: 
$$Q^*(s,a)=E_{(s',a') \sim p(.| s,a), \pi^*(.|s')}[r(s,a)+\gamma Q^{\pi^*}(s',a')]$$
$$=E_{s' \sim p(.| s,a)} \ E_{a' \sim \pi^*(.|s')}[r(s,a)+\gamma Q^*(s',a')]$$
$$=E_{s' \sim p(.| s,a)}[r(s,a)+\gamma E_{a' \sim \pi^*(.|s')}Q^*(s',a')]$$
$$=E_{s' \sim p(.| s,a)}[r(s,a)+\gamma max_{a'}Q^*(s',a')]$$
- The equation above has only one solution which is $Q^*(s,a)$. Therefore, it is plausible to find Q(s,a) by minimizing the distance between the right and left side in the equation for the optimal solution, which is represented by the loss $\mathcal{L}(\theta)$ ($\theta$ is the parameter of the model used to compute Q).

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
import random 
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # m can be an array  [s_, n_s_, a_, r_, game_over_]
        if len(self.memory)<self.max_memory:
            self.memory.append(m)
        else: 
            self.memory[:-1]=self.memory[1:]
            self.memory[-1]=m
    def random_access(self,num=1):
        if num==0 or len(self.memory)==0: 
            return 
        elif num==1: 
            return self.memory[np.random.randint(0,len(self.memory))]
        elif num>len(self.memory): 
            # take multiple indices with replacement(bootstrap)
            ind=np.random.randint(0,len(self.memory),(num,))
            temp_array=np.array(self.memory)
            return temp_array[ind]
        else: 
            return random.sample(self.memory,num)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[np.newaxis,:,:,:]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # sample a batch from memory 
        transition_batch=self.memory.random_access(self.batch_size)
        # fill arrays 
        for i in range(self.batch_size):
            s_i, n_s_i, a_i, r_i, game_over_=transition_batch[i]
            target_q[i]=self.model.predict(s_i[np.newaxis,:,:,:])
            input_states[i]=s_i

            if game_over_:
                target_q[i,a_i]=r_i
            else:
                target_q[i,a_i]=r_i+self.discount*np.max(self.model.predict(n_s_i[np.newaxis,:,:,:]))
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # MLP Model
        model=Sequential()
        model.add(Flatten())
        model.add(Dense(20,activation='relu'))
        model.add(Dense(20,activation='relu'))
        model.add(Dense(4))        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/050 | Loss 0.0089 | Win/lose count 3.5/6.0 (-2.5)
Epoch 001/050 | Loss 0.0111 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/050 | Loss 0.0177 | Win/lose count 5.0/5.0 (0.0)
Epoch 003/050 | Loss 0.0117 | Win/lose count 5.0/5.0 (0.0)
Epoch 004/050 | Loss 0.0061 | Win/lose count 5.0/6.0 (-1.0)
Epoch 005/050 | Loss 0.0058 | Win/lose count 7.0/6.0 (1.0)
Epoch 006/050 | Loss 0.0235 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/050 | Loss 0.0114 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/050 | Loss 0.0162 | Win/lose count 7.0/6.0 (1.0)
Epoch 009/050 | Loss 0.0133 | Win/lose count 6.0/4.0 (2.0)
Epoch 010/050 | Loss 0.0067 | Win/lose count 3.0/3.0 (0.0)
Epoch 011/050 | Loss 0.0156 | Win/lose count 4.5/6.0 (-1.5)
Epoch 012/050 | Loss 0.0481 | Win/lose count 5.0/5.0 (0.0)
Epoch 013/050 | Loss 0.0698 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/050 | Loss 0.0059 | Win/lose count 2.5/1.0 (1.5)
Epoch 015/050 | Loss 0.0055 | Win/lose count 2.5/8.0 (-5.5)
Epoch 016/050 | Loss 0.0052 | Win/los

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)

        model=Sequential()
        model.add(Conv2D(32, (2, 2),input_shape=(5, 5, self.n_state),padding='same',activation='relu'))
        model.add(Conv2D(16, (3, 3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))    
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0060 | Win/lose count 4.5/6.0 (-1.5)
Epoch 001/050 | Loss 0.0074 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/050 | Loss 0.0032 | Win/lose count 3.5/2.0 (1.5)
Epoch 003/050 | Loss 0.0057 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/050 | Loss 0.0021 | Win/lose count 1.5/1.0 (0.5)
Epoch 005/050 | Loss 0.0075 | Win/lose count 5.0/1.0 (4.0)
Epoch 006/050 | Loss 0.0016 | Win/lose count 6.5/6.0 (0.5)
Epoch 007/050 | Loss 0.0495 | Win/lose count 5.5/3.0 (2.5)
Epoch 008/050 | Loss 0.0039 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/050 | Loss 0.0025 | Win/lose count 4.5/5.0 (-0.5)
Epoch 010/050 | Loss 0.0625 | Win/lose count 4.0/0 (4.0)
Epoch 011/050 | Loss 0.0026 | Win/lose count 1.5/0 (1.5)
Epoch 012/050 | Loss 0.0032 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/050 | Loss 0.0012 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/050 | Loss 0.0025 | Win/lose count 6.0/3.0 (3.0)
Epoch 015/050 | Loss 0.0492 | Win/lose count 5.5/1.0 (4.5)
Epoch 016/050 | Loss 0.0018 | Win/lose count 11.0/3.0 (8

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/3.0. Average score (-1.0)
Win/lose count 5.0/3.0. Average score (0.5)
Win/lose count 4.0/4.0. Average score (0.3333333333333333)
Win/lose count 6.0/4.0. Average score (0.75)
Win/lose count 2.0/0. Average score (1.0)
Win/lose count 5.0/2.0. Average score (1.3333333333333333)
Win/lose count 7.0/5.0. Average score (1.4285714285714286)
Win/lose count 2.5/3.0. Average score (1.1875)
Win/lose count 5.5/2.0. Average score (1.4444444444444444)
Win/lose count 5.5/5.0. Average score (1.35)
Final score: 1.35
Test of the FC
Win/lose count 1.0/4.0. Average score (-3.0)
Win/lose count 3.0/0. Average score (0.0)
Win/lose count 3.0/5.0. Average score (-0.6666666666666666)
Win/lose count 5.5/1.0. Average score (0.625)
Win/lose count 1.0/3.0. Average score (0.1)
Win/lose count 1.0/0. Average score (0.25)
Win/lose count 2.5/1.0. Average score (0.42857142857142855)
Win/lose count 4.0/3.0. Average score (0.5)
Win/lose count 2.5/5.0. Average score (0.16666666666666666)
Win

In [0]:
HTML(display_videos('cnn_test9.mp4'))

In [0]:
HTML(display_videos('fc_test9.mp4'))

**Answer 9:** 
- First, we notice that the agent tends to be stuck in areas where there is no red squares: when the 5x5 patch around the agent does not contain a red square (+0.5) the agent can't make the right decision. In this case, we might notice some oscillations, because the agent wants to avoid the blue squares but has no idea where to go.  
- The second problem is the that the agent often gets stuck near the edge of the board, where he eats all the cheese and poison. Therefore, the patch that surrounds him quickly becomes empty, which causes oscillations.  
- The problem is more important with a lower temperature (thus we get a smaller number of non-empty squares). If we train the model in this case, the performance will get worse since the training samples become irrelevant (oscillations in empty areas). Moreover, another reason can be the overlap between 0.5 and -1 , since it is taken as 0.5 when creating the board. Hence, when the temperature increases, the overlap also does and the proportion positive/negative samples increases, which creates a bias. Therefore, when we test a model that was trained on a low temperature, in an environment of low temperature, we get a low score.
- The problem of oscillations might be solved by adding more exploration to the agent. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
     # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        # decreasing epsilon-greedy policy
        agent.set_epsilon(1-(0.95/49)*e)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.board[self.x,self.y] = 0
        self.t = 0

       
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0187 | Win/lose count 8.5/30.30000000000007 (-21.80000000000007)
Epoch 001/050 | Loss 0.0236 | Win/lose count 8.5/33.800000000000054 (-25.300000000000054)
Epoch 002/050 | Loss 0.0145 | Win/lose count 12.0/27.10000000000012 (-15.100000000000119)
Epoch 003/050 | Loss 0.0063 | Win/lose count 7.5/25.300000000000022 (-17.800000000000022)
Epoch 004/050 | Loss 0.0055 | Win/lose count 13.5/32.60000000000005 (-19.10000000000005)
Epoch 005/050 | Loss 0.0039 | Win/lose count 8.0/30.40000000000006 (-22.40000000000006)
Epoch 006/050 | Loss 0.0063 | Win/lose count 11.5/21.400000000000052 (-9.900000000000052)
Epoch 007/050 | Loss 0.0488 | Win/lose count 12.5/24.100000000000033 (-11.600000000000033)
Epoch 008/050 | Loss 0.0061 | Win/lose count 9.5/21.20000000000005 (-11.700000000000049)
Epoch 009/050 | Loss 0.0071 | Win/lose count 14.0/27.300000000000086 (-13.300000000000086)
Epoch 010/050 | Loss 0.0050 | Win/lose count 10.5/28.600000000000072 (-18.100000000000072)
Epoch 011/050

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 19.5/1.0. Average score (18.5)
Win/lose count 13.0/0. Average score (15.75)
Win/lose count 20.0/3.0. Average score (16.166666666666668)
Win/lose count 15.5/4.0. Average score (15.0)
Win/lose count 20.0/1.0. Average score (15.8)
Win/lose count 27.0/3.0. Average score (17.166666666666668)
Win/lose count 17.0/3.0. Average score (16.714285714285715)
Win/lose count 20.0/3.0. Average score (16.75)
Win/lose count 21.0/4.0. Average score (16.77777777777778)
Win/lose count 23.0/3.0. Average score (17.1)
Final score: 17.1


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
class Mimick(Agent):
    def __init__(self, grid_size,  epsilon = 0, memory_size=100, batch_size = 16,n_state=3):
        super(Mimick, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[np.newaxis,:,:,:]))

    def reinforce_mimick(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_a = np.zeros((self.batch_size, 4))
        # sample a batch from memory 
        transition_batch=self.memory.random_access(self.batch_size)
        # fill arrays 
        for i in range(self.batch_size):
            s_i, n_s_i, a_i, r_i, game_over_=transition_batch[i]
            input_states[i]=s_i
            target_a[i,a_i]=1
        
        l = self.model.train_on_batch(input_states, target_a)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class Mimick_CNN(Mimick):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(Mimick_CNN, self).__init__(*args,**kwargs)
        
        model=Sequential()
        model.add(Conv2D(32, (2, 2),input_shape=(5, 5, self.n_state),padding='same',activation='relu'))
        
        model.add(Conv2D(16, (3, 3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4,activation='softmax')) 
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0),  loss='categorical_crossentropy')
        self.model = model

In [0]:
def train_mimick(expert_agent,mim,env,epochs,prefix=''):
    # mimick is not epsilon greedy 
    mim.set_epsilon=0
    loss = 0

    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        # decreasing epsilon-greedy policy
        while not game_over:
            # The agent performs an action
            action = expert_agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            # Apply the reinforcement strategy
            loss = mim.reinforce_mimick(prev_state, state,  action, reward, game_over)
        print("Epoch {:03d}/{:03d} | Loss {:.4f} "
              .format(e, epochs, loss))
        mim.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
mim=Mimick_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
epochs_train_mim=200
train_mimick(agent,mim,env,epochs_train_mim,prefix='cnn_train_mimick')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 000/200 | Loss 1.1367 
Epoch 001/200 | Loss 0.9734 
Epoch 002/200 | Loss 0.7994 
Epoch 003/200 | Loss 0.9943 
Epoch 004/200 | Loss 1.0169 
Epoch 005/200 | Loss 0.6240 
Epoch 006/200 | Loss 0.6199 
Epoch 007/200 | Loss 0.6938 
Epoch 008/200 | Loss 0.6628 
Epoch 009/200 | Loss 0.5601 
Epoch 010/200 | Loss 0.5257 
Epoch 011/200 | Loss 0.5106 
Epoch 012/200 | Loss 0.5295 
Epoch 013/200 | Loss 0.8044 
Epoch 014/200 | Loss 0.6494 
Epoch 015/200 | Loss 0.6845 
Epoch 016/200 | Loss 0.6643 
Epoch 017/200 | Loss 0.4998 
Epoch 018/200 | Loss 0.6745 
Epoch 019/200 | Loss 0.7952 
Epoch 020/200 | Loss 0.7984 
Epoch 021/200 | Loss 0.7572 
Epoch 022/200 | Loss 0.7261 
Epoch 023/200 | Loss 0.6550 
Epoch 024/200 | Loss 0.5542 
Epoch 025/200 | Loss 0.5500 
Epoch 026/200 | Loss 0.6894 
Epoch 027/200 | Loss 0.7379 
Epoch 028/200 | Loss 0.6764 
Epoch 029/200 | Loss 0.4686 
Epoch 030/200 | Loss 0.8483 
Epoch 

In [0]:
# Evaluation
test(mim,env,epochs_test,prefix='cnn_test_mimick')
HTML(display_videos('cnn_test_mimick9.mp4'))

Win/lose count 21.0/3.0. Average score (18.0)
Win/lose count 16.0/2.0. Average score (16.0)
Win/lose count 28.0/4.0. Average score (18.666666666666668)
Win/lose count 12.5/2.0. Average score (16.625)
Win/lose count 27.0/4.0. Average score (17.9)
Win/lose count 19.0/1.0. Average score (17.916666666666668)
Win/lose count 18.0/5.0. Average score (17.214285714285715)
Win/lose count 13.5/2.0. Average score (16.5)
Win/lose count 20.5/5.0. Average score (16.38888888888889)
Win/lose count 25.0/4.0. Average score (16.85)
Final score: 16.85


**Answer BONUS:** We notice that the mimick has a performance that is good (score of 16.85). However, it failed to surpass the expert agent that was trained with exploration. This is natural since the mimick is learning from the agent. It can only hope to achieve the same levels of precision. Since at the end of the training of the mimick, we still have a non-zero loss, we can conclude that the mimick still does not do the exact actions dictated by the expert agent, and it will have an inferior performance. 

***